In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP
@pyimport numpy

In [ ]:
@pyimport sklearn.metrics as metrics
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
analyzer = "chuyu"

In [ ]:
data_info_all = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
function get_NMF_mask(cell_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full, threshold = 0.1)

    W = 32
    H = 32
    for i = cell_index
        which_z = z_all[i]
        
        roi_x1 = roi_x_idx_all[i]
        roi_x2 = min(img_width, roi_x1+W-1)
        roi_x2 == img_width ? roi_x1 = img_width-W+1 : roi_x1 = roi_x1 
        roi_y1 = roi_y_idx_all[i]
        roi_y2 = min(img_height, roi_y1+H-1)
        roi_y2 == img_height ? roi_y1 = img_height-H+1 : roi_y1 = roi_y1 
    
    
        cur_img_x = round(Int64, roi_centroid_x_all[i] + roi_x1 -1)
        cur_img_y = round(Int64,roi_centroid_y_all[i] + roi_y1 -1)
        footprint = S_all[:, i]
        footprint ./= maximum(footprint)
        mask = footprint .>= threshold
    
        # M.= 0
        # M[mask] .= 1
        
        M = reshape(footprint, (32, 32))
        
        M_full[roi_x1:roi_x2, roi_y1:roi_y2, which_z] .= M; 
    end

    return M_full
end

In [ ]:
which_data = 2
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]


experimenter = data_info[end]

ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

In [ ]:
NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
X_all = HDF5.readmmap(NMF_file["X_all"])
Y_all = HDF5.readmmap(NMF_file["Y_all"])
Z_all = HDF5.readmmap(NMF_file["Z_all"])
close(NMF_file)

In [ ]:
file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_cell_index = HDF5.readmmap(file["place_cell_index"])
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
close(file)

n_neuron = size(place_map_all_1, 3)

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end

tel_place_cell_index = intersect(place_cell_index, findall(whether_tel));

In [ ]:
# whether individual roi belongs to a certain region
anatomy_PF_correlation_filename = joinpath(data_path(ds_save_cy_1), "anatomy_PF_correlation_2d.h5")
anatomy_PF_correlation_file = h5open(anatomy_PF_correlation_filename, "r")
corr_map_pairwise_all_1 = read(anatomy_PF_correlation_file, "corr_map_pairwise_all_1")
anatomy_pairwise_distance = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance")
anatomy_pairwise_distance_z = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance_z")
close(anatomy_PF_correlation_file)

anatomy_pairwise_distance_copy = copy(anatomy_pairwise_distance)
for i in 1:size(anatomy_pairwise_distance_copy, 1)
    anatomy_pairwise_distance_copy[i,i] = NaN
end

corr_map_pairwise_all_1_copy = copy(corr_map_pairwise_all_1)
for i in 1:size(corr_map_pairwise_all_1_copy, 1)
    corr_map_pairwise_all_1_copy[i,i] = NaN
end

In [ ]:
NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
file = h5open(NMF_filename, "r")
ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
S_all = HDF5.readmmap(file["S_all"]) # roi mask
A_all = HDF5.readmmap(file["A_all"]) # roi activity
roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
z_all = HDF5.readmmap(file["z_all"]) # z for all rois
centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
close(file)

In [ ]:
img_width = size(ds_stack_mean, 1)
img_height = size(ds_stack_mean, 2)
n_z = size(ds_stack_mean, 3)
n_sweeps = size(A_all, 1)

In [ ]:
pair_list = intersect(findall(anatomy_pairwise_distance_copy.<5))

In [ ]:
# which_pair = numpy.random.choice(1:length(pair_list))
which_pair = 23
parent_cell_index = tel_place_cell_index[pair_list[which_pair][1]]
child_cell_index = tel_place_cell_index[pair_list[which_pair][2]]

parent_roi_index = findall(neuron_label.==parent_cell_index)
M_full_parent = zeros(Float32, img_width, img_height, n_z)
get_NMF_mask(parent_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_parent, 0);


child_roi_index = findall(neuron_label.==child_cell_index)
M_full_child = zeros(Float32, img_width, img_height, n_z)
get_NMF_mask(child_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_child, 0);

sum(M_full_parent.*M_full_child).>0

In [ ]:
z_all_parent = z_all[parent_roi_index]
z_all_child = z_all[child_roi_index];

z_all_union = union(z_all_parent, z_all_child);
z_mid = floor(Int32, mean(z_all_union))

In [ ]:
z_all_union

In [ ]:
figure(dpi=250)

subplot(1,2,1)
imshow(ds_stack_mean[:,:,z_mid]')
xlim(600, size(ds_stack_mean, 1))
ylim(100, 400)


subplot(1,2,2)
# imshow(nanmaximum(ds_stack_mean, dims=3)[:,:,1]')
imshow(ds_stack_mean[:,:,z_mid]')
imshow(nanmaximum(M_full_parent[:,:,z_all_union], dims=3)[:,:,1]', alpha = 0.3, cmap="Greens")
imshow(nanmaximum(M_full_child[:,:,z_all_union], dims=3)[:,:,1]', alpha = 0.3, cmap="Reds")
xlim(600, size(ds_stack_mean, 1))
ylim(100, 400)



In [ ]:
pair_list = intersect(findall(anatomy_pairwise_distance_copy.>10),findall(anatomy_pairwise_distance_copy.<12))

In [ ]:
mask_overlap = fill(false, size(anatomy_pairwise_distance))
@showprogress for which_pair in 1:length(pair_list)
        parent_cell_index = tel_place_cell_index[pair_list[which_pair][1]]
        child_cell_index = tel_place_cell_index[pair_list[which_pair][2]]
        
        parent_roi_index = findall(neuron_label.==parent_cell_index)
        M_full_parent = zeros(Float32, img_width, img_height, n_z)
        get_NMF_mask(parent_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_parent, 0);
        
        
        child_roi_index = findall(neuron_label.==child_cell_index)
        M_full_child = zeros(Float32, img_width, img_height, n_z)
        get_NMF_mask(child_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_child, 0);

        mask_overlap[pair_list[which_pair]] = sum(M_full_parent.*M_full_child).>0


        # if sum(M_full_parent.*M_full_child).==0
        #     println(pair_list[which_pair])
        #     println(which_pair)
        # end

        GC.gc()
end

In [ ]:
sum(mask_overlap)

In [ ]:
# which_pair = numpy.random.choice(1:length(pair_list))
which_pair = 23
parent_cell_index = tel_place_cell_index[pair_list[which_pair][1]]
child_cell_index = tel_place_cell_index[pair_list[which_pair][2]]

parent_roi_index = findall(neuron_label.==parent_cell_index)[3]
M_full_parent = zeros(Float32, img_width, img_height, n_z)
get_NMF_mask(parent_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_parent, 0);


child_roi_index = findall(neuron_label.==child_cell_index)[1]
M_full_child = zeros(Float32, img_width, img_height, n_z)
get_NMF_mask(child_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_child, 0);



In [ ]:
z_all_parent = z_all[parent_roi_index]


In [ ]:
z_all_child = z_all[child_roi_index]

In [ ]:
z_all_union = union(z_all_parent, z_all_child);
z_mid = floor(Int32, mean(z_all_union))

In [ ]:
figure(dpi=250)

subplot(1,2,1)
imshow(ds_stack_mean[:,:,z_mid]')
xlim(600, size(ds_stack_mean, 1))
ylim(100, 400)


subplot(1,2,2)
# imshow(nanmaximum(ds_stack_mean, dims=3)[:,:,1]')
imshow(ds_stack_mean[:,:,z_mid]')
imshow(nanmaximum(M_full_parent[:,:,z_all_union], dims=3)[:,:,1]', alpha = 0.3, cmap="Greens")
imshow(nanmaximum(M_full_child[:,:,z_all_union], dims=3)[:,:,1]', alpha = 0.3, cmap="Reds")
xlim(600, size(ds_stack_mean, 1))
ylim(100, 400)



In [ ]:
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]


experimenter = data_info[end]

ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

server_alternative = 25
ds_save_1_alternative = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_alternative)" ? "/data" : "/nfs/data$(server_alternative)")


NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
X_all = HDF5.readmmap(NMF_file["X_all"])
Y_all = HDF5.readmmap(NMF_file["Y_all"])
Z_all = HDF5.readmmap(NMF_file["Z_all"])
close(NMF_file)


file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_cell_index = HDF5.readmmap(file["place_cell_index"])
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
close(file)

n_neuron = size(place_map_all_1, 3)

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end

tel_place_cell_index = intersect(place_cell_index, findall(whether_tel));

@pyimport sklearn.metrics as metrics


# whether individual roi belongs to a certain region
anatomy_PF_correlation_filename = joinpath(data_path(ds_save_cy_1), "anatomy_PF_correlation_2d.h5")
anatomy_PF_correlation_file = h5open(anatomy_PF_correlation_filename, "r")
corr_map_pairwise_all_1 = read(anatomy_PF_correlation_file, "corr_map_pairwise_all_1")
anatomy_pairwise_distance = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance")
anatomy_pairwise_distance_z = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance_z")
close(anatomy_PF_correlation_file)

anatomy_pairwise_distance_copy = copy(anatomy_pairwise_distance)
for i in 1:size(anatomy_pairwise_distance_copy, 1)
    anatomy_pairwise_distance_copy[i,i] = NaN
end

corr_map_pairwise_all_1_copy = copy(corr_map_pairwise_all_1)
for i in 1:size(corr_map_pairwise_all_1_copy, 1)
    corr_map_pairwise_all_1_copy[i,i] = NaN
end

try
    NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end
try
    NMF_filename = joinpath(data_path(ds_save_1), "NMF_no_kappa.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end

try
    NMF_filename = joinpath(data_path(ds_save_1_alternative), "NMF.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end
try
    NMF_filename = joinpath(data_path(ds_save_1_alternative), "NMF_no_kappa.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end
    


img_width = size(ds_stack_mean, 1)
img_height = size(ds_stack_mean, 2)
n_z = size(ds_stack_mean, 3)
n_sweeps = size(A_all, 1)


pair_list = intersect(findall(anatomy_pairwise_distance_copy.<10))


mask_overlap = fill(false, size(anatomy_pairwise_distance))
@showprogress for which_pair in 1:length(pair_list)
        parent_cell_index = tel_place_cell_index[pair_list[which_pair][1]]
        child_cell_index = tel_place_cell_index[pair_list[which_pair][2]]
        
        parent_roi_index = findall(neuron_label.==parent_cell_index)
        M_full_parent = zeros(Float32, img_width, img_height, n_z)
        get_NMF_mask(parent_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_parent, 0);
        
        
        child_roi_index = findall(neuron_label.==child_cell_index)
        M_full_child = zeros(Float32, img_width, img_height, n_z)
        get_NMF_mask(child_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_child, 0);

        mask_overlap[pair_list[which_pair]] = sum(M_full_parent.*M_full_child).>0

        GC.gc()
end


h5open(joinpath(data_path(ds_save_analyzer_1), "anatomy_PF_correlation_2d.h5"), "r+") do file
    if haskey(file, "mask_overlap")
        delete_object(file, "mask_overlap")
    end

    file["mask_overlap"] = mask_overlap
end;

end

In [ ]:
for which_data = 1:length(data_info_all)

data_info = data_info_all[which_data]
    println(data_info)
experiment_filename_1 = data_info[1]
server_1 = data_info[2]


experimenter = data_info[end]

ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

server_alternative = 25
ds_save_1_alternative = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_alternative)" ? "/data" : "/nfs/data$(server_alternative)")


NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
X_all = HDF5.readmmap(NMF_file["X_all"])
Y_all = HDF5.readmmap(NMF_file["Y_all"])
Z_all = HDF5.readmmap(NMF_file["Z_all"])
close(NMF_file)


file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_cell_index = HDF5.readmmap(file["place_cell_index"])
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
close(file)

n_neuron = size(place_map_all_1, 3)

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end

tel_place_cell_index = intersect(place_cell_index, findall(whether_tel));

@pyimport sklearn.metrics as metrics


# whether individual roi belongs to a certain region
anatomy_PF_correlation_filename = joinpath(data_path(ds_save_cy_1), "anatomy_PF_correlation_2d.h5")
anatomy_PF_correlation_file = h5open(anatomy_PF_correlation_filename, "r")
corr_map_pairwise_all_1 = read(anatomy_PF_correlation_file, "corr_map_pairwise_all_1")
anatomy_pairwise_distance = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance")
anatomy_pairwise_distance_z = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance_z")
close(anatomy_PF_correlation_file)

anatomy_pairwise_distance_copy = copy(anatomy_pairwise_distance)
for i in 1:size(anatomy_pairwise_distance_copy, 1)
    anatomy_pairwise_distance_copy[i,i] = NaN
end

corr_map_pairwise_all_1_copy = copy(corr_map_pairwise_all_1)
for i in 1:size(corr_map_pairwise_all_1_copy, 1)
    corr_map_pairwise_all_1_copy[i,i] = NaN
end

try
    NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end
try
    NMF_filename = joinpath(data_path(ds_save_1), "NMF_no_kappa.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end

try
    NMF_filename = joinpath(data_path(ds_save_1_alternative), "NMF.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end
try
    NMF_filename = joinpath(data_path(ds_save_1_alternative), "NMF_no_kappa.h5")
    file = h5open(NMF_filename, "r")
    global ds_stack_mean = HDF5.readmmap(file["stack_mean"]) # reference images
    global S_all = HDF5.readmmap(file["S_all"]) # roi mask
    global A_all = HDF5.readmmap(file["A_all"]) # roi activity
    global roi_centroid_x_all = HDF5.readmmap(file["roi_centroid_x_all"]) # centroid x in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_centroid_y_all = HDF5.readmmap(file["roi_centroid_y_all"]) # centroid y in the 32*32 image (for the square, the centroid is located by the mask)
    global roi_x_idx_all = HDF5.readmmap(file["roi_x_idx_all"]) # roi x (for the square, the centroid is located by the mask)
    global roi_y_idx_all = HDF5.readmmap(file["roi_y_idx_all"]) # roi y (for the square, the centroid is located by the mask)
    global z_all = HDF5.readmmap(file["z_all"]) # z for all rois
    global centroid_x_all = HDF5.readmmap(file["centroid_x_all"]) # centroid x
    global centroid_y_all = HDF5.readmmap(file["centroid_y_all"]) # centroid y
    close(file)
catch end
    


img_width = size(ds_stack_mean, 1)
img_height = size(ds_stack_mean, 2)
n_z = size(ds_stack_mean, 3)
n_sweeps = size(A_all, 1)


pair_list = intersect(findall(anatomy_pairwise_distance_copy.<5))


mask_overlap = fill(false, size(anatomy_pairwise_distance))
@showprogress for which_pair in 1:length(pair_list)
        parent_cell_index = tel_place_cell_index[pair_list[which_pair][1]]
        child_cell_index = tel_place_cell_index[pair_list[which_pair][2]]
        
        parent_roi_index = findall(neuron_label.==parent_cell_index)
        M_full_parent = zeros(Float32, img_width, img_height, n_z)
        get_NMF_mask(parent_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_parent, 0);
        
        
        child_roi_index = findall(neuron_label.==child_cell_index)
        M_full_child = zeros(Float32, img_width, img_height, n_z)
        get_NMF_mask(child_roi_index, img_width, img_height, roi_x_idx_all, roi_y_idx_all, roi_centroid_x_all, roi_centroid_y_all, S_all, z_all, M_full_child, 0);

        mask_overlap[pair_list[which_pair]] = sum(M_full_parent.*M_full_child).>0
        if sum(M_full_parent.*M_full_child).==0
            println(pair_list[which_pair])
            println(which_pair)
        end
        GC.gc()
end




end